# Using the lmt-analysis library to export detection and event tables to Excel
---

### Import required packages

In [ ]:
import os
import sys
import numpy
import pandas
import sqlite3

sys.path.insert(1, "../")

from lmtanalysis import Measure
from lmtanalysis import Features
from lmtanalysis.Util     import getAllEvents
from lmtanalysis.Animal   import AnimalPool
from lmtanalysis.Event    import EventTimeLine
from lmtanalysis.FileUtil import getFilesToProcess

### Load an SQLite database

In [ ]:
files = getFilesToProcess()

if files and len(files) > 0:
    SQLITE_FN = files[0]
    out_dir  = os.path.dirname(SQLITE_FN)
    out_base = os.path.splitext(SQLITE_FN)[0]
else: print("No file selected...")

### Connect to data base and load detections for first hour

In [ ]:
connection = sqlite3.connect( SQLITE_FN )

animalPool = AnimalPool()

# load infos about the animals
animalPool.loadAnimals( connection )

# load all detection (positions) of all animals for the first hour
animalPool.loadDetection( start = 0, end = Measure.oneHour)

# retrieve all event names
all_events = getAllEvents(connection=connection)



### Compute detection summary table for two different time ranges
 * 12 time bins: 0 to 60 minutes with 5 minutes long interval
 * 1 time bin  : 0 to 60 minutes with 60 minutes long interval

In [ ]:
detections_5min  = Features.computeDetectionFeatures(animalPool, start="0min", end="60min", freq="5min")
detections_60min = Features.computeDetectionFeatures(animalPool, start="0min", end="60min", freq="60min")

# show
detections_5min

### Compute event summary table for two different time ranges
 * 12 time bins: 0 to 60 minutes with 5 minutes long interval
 * 1 time bin  : 0 to 60 minutes with 60 minutes long interval

In [ ]:
events_5min     = Features.computeEventFeatures(animalPool, start="0min", end="60min", freq="5min")
events_60min    = Features.computeEventFeatures(animalPool, start="0min", end="60min", freq="60min")

# show one table
events_60min

### Export to Excel file
stored in same folder as sqlite file, with 4 sheets:
 * Detection 5min
 * Detection 60min
 * Events 5min
 * Events 60min

In [ ]:
export = {}
export["Detection 5min"]  = detections_5min
export["Detection 60min"] = detections_60min

export["Events 5min"]     = events_5min
export["Events 60min"]    = events_60min

with pandas.ExcelWriter(out_base + ".xlsx") as excl_writer:
    for sheet_name, sheet_df in export.items():
        sheet_df.to_excel(excl_writer, sheet_name=sheet_name)